In [1]:
import os

os.environ['OPENAI_API_KEY'] = '****'
os.environ['LANGCHAIN_API_KEY'] = '****'
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = '08-05'

In [4]:
!pip install -qU langchain_community

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.4.0 requires tokenizers<0.20.0,>=0.19.1, but you have tokenizers 0.21.0 which is incompatible.


In [9]:
texts = [
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. ",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]


#### 지원되는 임베딩 모델 확인

- https://ollama.com/library

In [5]:
from langchain_community.embeddings import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    # model="chatfire/bge-m3:q8_0" # BGE-M3
)

In [7]:
# 쿼리 임베딩
embedded_query = ollama_embeddings.embed_query("LangChain 에 대해서 상세히 알려주세요.")
# 임베딩 차원 출력
len(embedded_query)


768

In [10]:
# 문서 임베딩
embedded_documents = ollama_embeddings.embed_documents(texts)


In [22]:
embedded_documents[0][:5]

[-0.24748356640338898,
 -0.23559220135211945,
 -2.8910884857177734,
 -0.6004036664962769,
 0.6054303050041199]

In [27]:
# import numpy as np

# # 질문(embedded_query): LangChain 에 대해서 알려주세요.
# similarity = np.array(embedded_query) @ np.array(embedded_documents).T

# # 유사도 기준 내림차순 정렬
# sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

# # 결과 출력
# print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
# for i, idx in enumerate(sorted_idx):
#     print(f"[{i}] 유사도: {similarity[idx]:.3f} | {texts[idx]}")
#     print()


In [26]:
import numpy as np

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
similarity = np.array(embedded_query) @ np.array(embedded_documents).T

# 유사도 기준 내림차순 정렬
sorted_idx = similarity.argsort()[::-1]

# 유사도 정규화 (최소값 0, 최대값 1로 변환)
similarity_min = similarity.min()
similarity_max = similarity.max()
normalized_similarity = (similarity - similarity_min) / (similarity_max - similarity_min)

# 결과 출력
print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] 유사도: {normalized_similarity[idx]:.3f} | {texts[idx]}")
    print()


[Query] LangChain 에 대해서 알려주세요.
[0] 유사도: 1.000 | LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.

[1] 유사도: 0.753 | 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 

[2] 유사도: 0.558 | LangChain simplifies the process of building applications with large language models

[3] 유사도: 0.550 | 안녕, 만나서 반가워.

[4] 유사도: 0.000 | Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.

